# 패키지 불러오기

In [ ]:
import numpy as np
import torch
import tensorflow as tf
from transformers import Trainer
from transformers import pipeline
from transformers import EvalPrediction
from transformers import TrainingArguments
from tokenization_kbalbert import KbAlbertCharTokenizer
from transformers import AlbertForSequenceClassification

# 디바이스 검사 및 설정

In [ ]:
# 디바이스 검사
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# 디바이스 설정    
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

# 토크나이저 설정

In [ ]:
tokenizer = KbAlbertCharTokenizer.from_pretrained('kb-albert-char')

# 데이터 로더

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=32)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=32)

In [ ]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, train_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

# 학습

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2.0,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=2000
)

In [ ]:
model = AlbertForSequenceClassification.from_pretrained('kb-albert-char', num_labels = 4)
model.cuda()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

# 평가

In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {
            'acc': (preds == p.label_ids).mean()
        }
trainer.compute_metrics = compute_metrics
trainer.evaluate(test_dataset)

# 파이프라인 만들기

In [ ]:
review_classifier = pipeline('review-analysis', 
                           model=model.cpu(), tokenizer=tokenizer, framework='pt')

# labels = 0: "칭찬", 1: "APP 버그", 2: "정정 요청", 3: "기타"